In [13]:
%pip install --user selenium pytest pytest-html webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

folders = [
    "Task2_AutomationTesting/tests",
    "Task2_AutomationTesting/pages",
    "Task2_AutomationTesting/utils",
    "Task2_AutomationTesting/reports",
    "Task2_AutomationTesting/screenshots"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

In [5]:
%%writefile Task2_AutomationTesting/conftest.py
import pytest
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

@pytest.fixture(scope="function")
def driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    driver.implicitly_wait(10)
    yield driver
    driver.quit()

Overwriting Task2_AutomationTesting/conftest.py


In [6]:
%%writefile Task2_AutomationTesting/pages/base_page.py
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class BasePage:
    def __init__(self, driver):
        self.driver = driver
        self.wait = WebDriverWait(driver, 10)

    def click(self, locator):
        self.wait.until(EC.element_to_be_clickable(locator)).click()

    def type(self, locator, text):
        el = self.wait.until(EC.visibility_of_element_located(locator))
        el.clear()
        el.send_keys(text)

    def get_text(self, locator):
        return self.wait.until(EC.visibility_of_element_located(locator)).text

Overwriting Task2_AutomationTesting/pages/base_page.py


In [7]:
%%writefile Task2_AutomationTesting/pages/login_page.py
from selenium.webdriver.common.by import By
from pages.base_page import BasePage

class LoginPage(BasePage):
    USERNAME = (By.NAME, "username")
    PASSWORD = (By.NAME, "password")
    LOGIN_BTN = (By.CSS_SELECTOR, "button[type='submit']")
    ERROR_MSG = (By.XPATH, "//p[contains(@class,'oxd-alert-content-text')]")

    def __init__(self, driver):
        super().__init__(driver)
        self.url = "https://opensource-demo.orangehrmlive.com"

    def open(self):
        self.driver.get(self.url)

    def login(self, username, password):
        self.type(self.USERNAME, username)
        self.type(self.PASSWORD, password)
        self.click(self.LOGIN_BTN)

    def get_error(self):
        return self.get_text(self.ERROR_MSG)

Overwriting Task2_AutomationTesting/pages/login_page.py


In [10]:
%%writefile Task2_AutomationTesting/pages/dashboard_page.py
from selenium.webdriver.common.by import By
from pages.base_page import BasePage

class DashboardPage(BasePage):
    DASHBOARD_HEADER = (By.XPATH, "//h6[text()='Dashboard']")

    def is_displayed(self):
        return self.get_text(self.DASHBOARD_HEADER) == "Dashboard"

Writing Task2_AutomationTesting/pages/dashboard_page.py


In [8]:
%%writefile Task2_AutomationTesting/tests/test_login.py
import pytest
from pages.login_page import LoginPage
from pages.dashboard_page import DashboardPage

class TestLogin:
    @pytest.fixture(autouse=True)
    def setup(self, driver):
        self.driver = driver
        self.login_page = LoginPage(driver)
        self.login_page.open()

    def test_valid_login(self):
        self.login_page.login("Admin", "admin123")
        dashboard = DashboardPage(self.driver)
        assert dashboard.is_displayed(), "Dashboard should be visible after login"

    def test_invalid_password(self):
        self.login_page.login("Admin", "wrongpass")
        assert "Invalid credentials" in self.login_page.get_error(), "Error message should appear"

Overwriting Task2_AutomationTesting/tests/test_login.py


In [9]:
%%writefile Task2_AutomationTesting/requirements.txt
selenium==4.15.2
pytest==7.4.3
pytest-html==4.1.1
webdriver-manager==4.0.1

Overwriting Task2_AutomationTesting/requirements.txt


In [10]:
%%writefile Task2_AutomationTesting/pytest.ini
[pytest]
addopts = -v -s --html=reports/test_report.html --self-contained-html
testpaths = tests
python_files = test_*.py
python_classes = Test*
python_functions = test_*

Overwriting Task2_AutomationTesting/pytest.ini


In [11]:
%%writefile Task2_AutomationTesting/README.md
# Automation Testing Project

## Project Overview
Automated UI testing for OrangeHRM Demo using Selenium, pytest, and Page Object Model.



## Setup Instructions
1. Install Python 3.8+
2. Install dependencies: `pip install -r requirements.txt`

## Running Tests
```bash
pytest tests/ -v
pytest tests/ --html=reports/test_report.html

Overwriting Task2_AutomationTesting/README.md


In [12]:
!pytest Task2_AutomationTesting/tests/ -v

ImportError while loading conftest 'C:\Users\cbp\Task2_AutomationTesting\conftest.py'.
Task2_AutomationTesting\conftest.py:4: in <module>
    from webdriver_manager.chrome import ChromeDriverManager
E   ModuleNotFoundError: No module named 'webdriver_manager'


In [17]:
!pytest Task2_AutomationTesting/tests/ --html=Task2_AutomationTesting/reports/test_report.html --self-contained-html

============================= test session starts =============================
platform win32 -- Python 3.13.5, pytest-8.3.4, pluggy-1.5.0 -- C:\ProgramData\anaconda3\python.exe
cachedir: .pytest_cache
metadata: {'Python': '3.13.5', 'Platform': 'Windows-11-10.0.26100-SP0', 'Packages': {'pytest': '8.3.4', 'pluggy': '1.5.0'}, 'Plugins': {'html': '4.1.1', 'metadata': '3.1.1', 'anyio': '4.7.0'}, 'JAVA_HOME': 'F:\\Program Files\\Java\\jdk-11.0.11'}
rootdir: C:\Users\cbp\Task2_AutomationTesting
configfile: pytest.ini
plugins: html-4.1.1, metadata-3.1.1, anyio-4.7.0
collecting ... collected 2 items

Task2_AutomationTesting\tests\test_login.py::TestLogin::test_valid_login PASSED
Task2_AutomationTesting\tests\test_login.py::TestLogin::test_invalid_password PASSED

- Generated html report: file:///C:/Users/cbp/Task2_AutomationTesting/reports/test_report.html -
============================= 2 passed in 19.87s ==============================


In [ ]:
!pytest Task2_AutomationTesting/tests/ --html=Task2_AutomationTesting/reports/test_report.html --self-contained-html